# 动态属性和特性

In [ ]:
# 数据的属性和处理属性的方法统称属性(attribute), 方法是可调用的属性
# 除此之外，还可以创建特性(property)

In [ ]:
from urllib.request import urlopen
import warnings
import os
import json

In [ ]:
URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)

In [10]:
from collections import abc

class FrozenJSON:

    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence)
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = dict(mapping)

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(cls.build(item) for item in obj)]
        else:
            return obj

In [11]:
# 原始数据中的键可能不适合作为属性名
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [13]:
# 可以用getattr(grad, 'class')
# "异常处理吧，没啥好的解决方法"
grad.class

SyntaxError: invalid syntax (<ipython-input-13-bb5c99ef29c5>, line 1)

In [ ]:
# 使用__new__方法以灵活的方式创建对象
# __new__是个类方法（使用特殊方法处理，因此不必使用@classmethod装饰器)
# 必须返回一个实例，返回的实例传给__init__，__init__禁止返回任何值

In [ ]:
# shelve（架子） 模块
# pickle（泡菜） 模块
# 泡菜放在架子上，所以shelve模块提供了pickle存储方式
import warnings
import shelve

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.updata(kwargs)

def load_db(db):
    raw_data = load()
    warnings.warn('loading '+ DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = Record(**record)

db = shelve.open(DB_NAME)
if CONFERENCE not in db:
    load_db(db)

speaker = db['speaker.3471']
print (type(speaker))
print (speaker.name, speaker.twitter)

In [ ]:
import warnings
import inspect

import osconfeed

DB_NAME = 'data/schedule2_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __eq__(self, other):
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

class MissingDatabaseError(RuntimeError):
    """need db but does not exist"""

class DbRecord(Record):
    __db = None

    @staticmethod
    def set_db(db):
        DbRecord.__db = db

    @staticmethod
    def get_db():
        return DbRecord.__db

    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise
                
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name  = self.__class__.__name__
            return '<{} serial = {!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()

class Event(DbRecord):
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)

    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'):
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self._speaker_objs = [fetch('speaker.{}'.format(key)) for key in spkr_serials]
        return self._speaker_objs

    def __repr__(self):
        if hasattr(self, 'name'):
            cls_name = self.__class__.__name
            # {!r} 反应本体
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()

In [ ]:
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, DbRecord)
        if inspect.isclass(cls) and issubclass(cls, DbRecord):
            factory = cls
        else:
            factory = DbRecord
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)

In [ ]:
# 使用特性验证属性
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

# 重量是负值时，金额小计为负值
# 数据属性换成特性
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    @property
    def weight(self):
        return self.__weight

    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

In [ ]:
# 特性全解析
# property构造方法: property(fget = None, fset = None, fdel = None, doc = None)
# “经典”写法
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    def get_weight(self):
        return self.__weight

    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    weight = property(get_weight, set_weight)

In [23]:
# 特性都是类属性，但是特性管理的其实是实例属性的存取
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [24]:
obj = Class()

In [25]:
vars(obj)

{}

In [26]:
obj.data = 'bar'
print (vars(obj))
print (obj.data)
print (Class.data)

{'data': 'bar'}
bar
the class data attr


In [27]:
print (Class.prop) # 注意，这时候prop是这个类的一个特性

In [28]:
print (obj.prop)
obj.prop = 'foo'

the prop value


AttributeError: can't set attribute

In [29]:
obj.__dict__['prop'] = 'foo'
print (vars(obj))
print (obj.prop)
Class.prop = 'baz' # 销毁特性
print (obj.prop)

{'data': 'bar', 'prop': 'foo'}
the prop value
foo


In [30]:
print (obj.data)
print (Class.data)
Class.data = property(lambda self: 'the "data" prop value')
print (obj.data)
del Class.data
print (obj.data)

bar
the class data attr
the "data" prop value
bar


In [42]:
# 定义一个特性工厂函数
def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]

    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')

    return property(qty_getter, qty_setter)

class LineItem:
    weight = quantity('weight')
    price = quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [ ]:
# 删除属性（不常用，了解下即可）
# del obj.attr
# @menber.deleter

In [ ]:
# 影响属性处理的重要属性和函数
# __class__
# obj.__class__ 和 type(obj)作用相同
# __dict__
# 一个映射，存储对象或类的可写属性
# __slots__
# 限制实例能拥有哪些属性，如果slots没有'__dict__',实例只允许有指定名称的属性

In [ ]:
# 处理属性的内置函数

In [44]:
# dir([object]) 列出对象的大多数属性
# getattr(object, name [, default])
# hasattr(object, name)
# setattr(object, name, value)
# vars([object]) 等同于obj.__dict__，如果没参数，vars()等同于locals()

In [53]:
dir(obj)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data',
 'prop']

In [54]:
obj.__dict__

{'data': 'bar', 'prop': 'foo'}